In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stratified/train.pt
/kaggle/input/stratified/val.pt
/kaggle/input/stratified/test.pt
/kaggle/input/random/train.pt
/kaggle/input/random/val.pt
/kaggle/input/random/test.pt
/kaggle/input/tox21data/tox21.csv


In [2]:
!pip install torch-geometric
!pip install rdkit-pypi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 54.2 MB/s eta 0:00:00:00:0100:01


In [3]:
import torch
import torch.nn as nn
from torch_geometric.data import Data
from torch.utils.data import DataLoader
from torch_geometric.nn import GATConv, global_mean_pool, global_max_pool, global_add_pool
from torch_geometric.explain import Explainer, GNNExplainer
from rdkit import Chem
from rdkit.Chem import Descriptors
import numpy as np


## Preprocessing

In [4]:
# import pandas as pd
# import numpy as np
# from rdkit import Chem
# import torch
# from torch_geometric.data import Data
# import random

# # ============================================================================
# # LOAD DỮ LIỆU
# # ============================================================================

# TOX21 = "/kaggle/input/tox21data/tox21.csv"
# df = pd.read_csv(TOX21)

# print('='*70)
# print('📊 LOAD DATA')
# print('='*70)
# print(df.head())

# label_cols = [
#     "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase",
#     "NR-ER", "NR-ER-LBD", "NR-PPAR-gamma",
#     "SR-ARE", "SR-ATAD5", "SR-HSE", "SR-MMP", "SR-p53"
# ]

# smiles_df = df["smiles"].tolist()
# mol_ids_df = df["mol_id"].tolist()
# labels_df = df[label_cols].values  # (n_samples, 12)

# print(f"\n✅ SMILES: {len(smiles_df)} molecules")
# print(f"✅ Labels shape: {labels_df.shape}")

# # ============================================================================
# # TẠO LABELS_CLEAN + MASK
# # ============================================================================

# labels_clean = np.nan_to_num(labels_df, nan=0.0)
# mask = (~np.isnan(labels_df)).astype(np.float32)

# print(f"✅ Labels_clean shape: {labels_clean.shape}")
# print(f"✅ Mask shape: {mask.shape}")

# # ============================================================================
# # BUILD DATASET
# # ============================================================================

# dataset = []
# failed = 0

# print(f"\n{'='*70}")
# print("🏗️  BUILD DATASET")
# print(f"{'='*70}")

# for i, smi in enumerate(smiles_df):
#     # Convert SMILES → Mol
#     try:
#         mol = Chem.MolFromSmiles(smi, sanitize=False)
#         Chem.SanitizeMol(mol)
#     except:
#         failed += 1
#         continue
    
#     if mol is None:
#         failed += 1
#         continue
    
#     # ===== NODE FEATURES =====
#     atom_features = []
#     periodic_table = Chem.GetPeriodicTable()
    
#     for atom in mol.GetAtoms():
#         atomic_num = atom.GetAtomicNum()
#         valence_electrons = periodic_table.GetNOuterElecs(atomic_num)
#         features = [
#             atomic_num,
#             atom.GetTotalValence(),
#             atom.GetTotalDegree(),
#             int(atom.GetIsAromatic()),
#             atom.GetFormalCharge(),
#             valence_electrons,           
#             atom.GetNumImplicitHs()      
#         ]
#         atom_features.append(features)
    
#     x = torch.tensor(atom_features, dtype=torch.float)
    
#     # ===== EDGE INDEX =====
#     edge_index = []
#     for bond in mol.GetBonds():
#         a = bond.GetBeginAtomIdx()
#         b = bond.GetEndAtomIdx()
#         edge_index.append([a, b])
#         edge_index.append([b, a])
    
#     if len(edge_index) == 0:
#         edge_index = torch.zeros((2, 0), dtype=torch.long)
#     else:
#         edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    
#     # ===== LABELS (GRAPH-LEVEL!) =====
#     y = torch.tensor(labels_clean[i], dtype=torch.float)        # (12,) ✅
#     m = torch.tensor(mask[i], dtype=torch.float)                # (12,) ✅
    
#     # ===== TẠO DATA OBJECT =====
#     data = Data(
#         x=x,
#         edge_index=edge_index,
#         y=y,
#         mask=m,
#         mol_id=mol_ids_df[i],
#         num_nodes=x.shape[0]
#     )
    
#     dataset.append(data)

# print(f"✅ Dataset created: {len(dataset)} molecules")
# print(f"❌ Failed to parse: {failed} SMILES")

# # Kiểm tra sample
# sample = dataset[0]
# print(f"\n📌 Sample Data Object:")
# print(f"   x.shape: {sample.x.shape}")
# print(f"   edge_index.shape: {sample.edge_index.shape}")
# print(f"   y.shape: {sample.y.shape} ✅ (phải là (12,))")
# print(f"   mask.shape: {sample.mask.shape} ✅ (phải là (12,))")
# print(f"   num_nodes: {sample.num_nodes}")
# print(f"   mol_id: {sample.mol_id}")

# # ============================================================================
# # SPLIT DATASET
# # ============================================================================

# print(f"\n{'='*70}")
# print("📊 SPLIT DATASET")
# print(f"{'='*70}")

# random.seed(42)
# random.shuffle(dataset)

# total = len(dataset)
# len_train = int(0.8 * total)
# len_val = int(0.1 * total)
# len_test = total - len_train - len_val

# print(f"Tổng số mẫu: {total}")
# print(f"Train: {len_train} (80%)")
# print(f"Val:   {len_val} (10%)")
# print(f"Test:  {len_test} (10%)")

# train_dataset = dataset[:len_train]
# val_dataset = dataset[len_train : len_train + len_val]
# test_dataset = dataset[len_train + len_val :]

# # ============================================================================
# # VERIFY
# # ============================================================================

# print(f"\n{'='*70}")
# print("✅ VERIFY DATASET")
# print(f"{'='*70}")

# print(f"\n📌 Train Sample:")
# print(f"   x.shape: {train_dataset[0].x.shape}")
# print(f"   y.shape: {train_dataset[0].y.shape} ✅")

# print(f"\n📌 Val Sample:")
# print(f"   x.shape: {val_dataset[0].x.shape}")
# print(f"   y.shape: {val_dataset[0].y.shape} ✅")

# print(f"\n📌 Test Sample:")
# print(f"   x.shape: {test_dataset[0].x.shape}")
# print(f"   y.shape: {test_dataset[0].y.shape} ✅")

# # ============================================================================
# # SAVE
# # ============================================================================

# print(f"\n{'='*70}")
# print("💾 SAVE DATASET")
# print(f"{'='*70}")

# torch.save(train_dataset, "train.pt")
# torch.save(val_dataset, "val.pt")
# torch.save(test_dataset, "test.pt")

# print("✅ train.pt saved")
# print("✅ val.pt saved")
# print("✅ test.pt saved")

# print(f"\n{'='*70}")
# print("✨ DONE!")
# print(f"{'='*70}")

In [5]:
train_dataset = torch.load("/kaggle/input/stratified/train.pt", weights_only=False)
val_dataset   = torch.load("/kaggle/input/stratified/val.pt", weights_only=False)
test_dataset  = torch.load("/kaggle/input/stratified/test.pt", weights_only=False)

In [6]:
# def clean_dataset(ds, dim=9):
#     clean = []
#     for d in ds:
#         if d.edge_attr is None:
#             continue
#         if d.edge_attr.size(1) != dim:
#             continue
#         clean.append(d)
#     return clean

# train_dataset = clean_dataset(train_dataset)
# val_dataset   = clean_dataset(val_dataset)
# test_dataset  = clean_dataset(test_dataset)

In [7]:
data = train_dataset[0]
print(data)
print("Node feature matrix (x):")
print(data.x)
print("x shape:", data.x.shape)


Data(x=[11, 41], edge_index=[2, 20], edge_attr=[20, 9], y=[12], mask=[12], mol_id='TOX9302', num_nodes=11)
Node feature matrix (x):
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.1201, 1.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.1201, 1.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000

In [8]:

def extract_graph_labels(batch, num_classes, device):
    """
    Extract and reshape labels from batch
    """
    y = batch.y
    
    # If already (batch_size, num_classes), return as is
    if len(y.shape) == 2 and y.shape[1] == num_classes:
        return y.to(device)
    
    # If (batch_size*num_classes,), reshape
    if len(y.shape) == 1:
        batch_size = y.shape[0] // num_classes
        y = y.reshape(batch_size, num_classes)
        return y.to(device)
    
    return y.to(device)

## GAT MODEL

In [9]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv, GATv2Conv, global_mean_pool, global_max_pool, global_add_pool
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, accuracy_score


class GAT(nn.Module):
    """
    State-of-the-art GAT cho multi-label classification với imbalanced data:
    
    Features:
    - GATv2Conv (better than GAT)
    - Multiple pooling strategies (mean + max + sum)
    - Batch Normalization
    - Residual connections at graph level
    - Dropout regularization
    - Better weight initialization
    """
    def __init__(self, input_dim=41, hidden_dim=512, num_heads=8,
                 num_layers=3, embedding_dim=1024, dropout=0.2, 
                 edge_dim=None, add_self_loops=True):
        super().__init__()
        
        self.num_layers = num_layers
        self.dropout = dropout
        
        # GAT Layers - using GATv2 (more expressive)
        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        
        # Input layer
        self.convs.append(
            GATv2Conv(
                input_dim, 
                hidden_dim, 
                heads=num_heads,
                dropout=dropout,
                concat=True,
                add_self_loops=add_self_loops,
                edge_dim=edge_dim
            )
        )
        self.batch_norms.append(nn.BatchNorm1d(hidden_dim * num_heads))
        
        # Hidden layers
        for i in range(num_layers - 2):
            self.convs.append(
                GATv2Conv(
                    hidden_dim * num_heads,
                    hidden_dim,
                    heads=num_heads,
                    dropout=dropout,
                    concat=True,
                    add_self_loops=add_self_loops,
                    edge_dim=edge_dim
                )
            )
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim * num_heads))
        
        # Final layer - average attention heads
        self.convs.append(
            GATv2Conv(
                hidden_dim * num_heads,
                hidden_dim,
                heads=num_heads,
                dropout=dropout,
                concat=False,  # Average instead of concat
                add_self_loops=add_self_loops,
                edge_dim=edge_dim
            )
        )
        self.batch_norms.append(nn.BatchNorm1d(hidden_dim))
        
        # Graph-level pooling projection
        # Concat mean + max + sum = 3 * hidden_dim
        pooling_dim = hidden_dim * 3
        
        # MLP for graph embedding
        self.graph_mlp = nn.Sequential(
            nn.Linear(pooling_dim, embedding_dim),
            nn.BatchNorm1d(embedding_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            
            nn.Linear(embedding_dim, embedding_dim),
            nn.BatchNorm1d(embedding_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout)
        )
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """Xavier initialization for better gradient flow"""
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.BatchNorm1d):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)
    
    def forward(self, x, edge_index, batch, edge_attr=None):
        """
        Args:
            x: Node features [num_nodes, input_dim]
            edge_index: Edge indices [2, num_edges]
            batch: Batch assignment [num_nodes]
            edge_attr: Edge features [num_edges, edge_dim] (optional)
        """
        
        # Message passing
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index, edge_attr=edge_attr)
            x = self.batch_norms[i](x)
            x = torch.relu(x)
            x = nn.functional.dropout(x, p=self.dropout, training=self.training)
        
        # Multiple pooling strategies
        x_mean = global_mean_pool(x, batch)
        x_max = global_max_pool(x, batch)
        x_sum = global_add_pool(x, batch)
        
        # Concatenate poolings
        graph_repr = torch.cat([x_mean, x_max, x_sum], dim=-1)
        
        # Graph-level MLP
        graph_embedding = self.graph_mlp(graph_repr)
        
        return graph_embedding


In [10]:
class MultiLabelClassifier(nn.Module):
    """
    Advanced classifier với temperature scaling và multiple hidden layers
    """
    def __init__(self, embedding_dim=512, num_classes=12, 
                 hidden_dim=256, dropout=0.3, temperature=1.0):
        super().__init__()
        
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            
            nn.Linear(hidden_dim // 2, num_classes)
        )
        
        # Temperature parameter for calibration
        self.temperature = nn.Parameter(torch.ones(1) * temperature)
        
        # Initialize
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
    
    def forward(self, x):
        logits = self.classifier(x)
        # Apply temperature scaling
        scaled_logits = logits / torch.clamp(self.temperature, min=0.5, max=2.0)
        return scaled_logits



In [11]:


class AsymmetricLossOptimized(nn.Module):
    """
    Asymmetric Loss specifically designed for imbalanced multi-label
   
    """
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, 
                 disable_torch_grad_focal_loss=True):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
    
    def forward(self, x, y):
        """
        Args:
            x: logits [batch_size, num_classes]
            y: targets [batch_size, num_classes] (0 or 1)
        """
        # Probabilities
        x_sigmoid = torch.sigmoid(x)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)

        # Basic CE
        los_pos = y * torch.log(xs_pos.clamp(min=1e-8))
        los_neg = (1 - y) * torch.log(xs_neg.clamp(min=1e-8))
        loss = los_pos + los_neg

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            
            pt0 = xs_pos * y
            pt1 = xs_neg * (1 - y)
            pt = pt0 + pt1
            one_sided_gamma = self.gamma_pos * y + self.gamma_neg * (1 - y)
            one_sided_w = torch.pow(1 - pt, one_sided_gamma)
            
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            
            loss *= one_sided_w

        return -loss.mean()



# Train

In [12]:
def _find_optimal_thresholds(all_probs, all_labels, num_classes):
    """Improved threshold finding using multiple strategies"""
    from sklearn.metrics import precision_recall_curve
    
    optimal_thresholds = []
    per_class_f1 = []
    
    for i in range(num_classes):
        probs = all_probs[:, i]
        labels = all_labels[:, i]
        
        # Skip if no positive samples
        if labels.sum() == 0:
            optimal_thresholds.append(0.5)
            per_class_f1.append(0.0)
            continue
            
        # Strategy 1: Precision-Recall curve (more robust)
        try:
            precision, recall, thresholds = precision_recall_curve(labels, probs)
            f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
            best_idx = np.argmax(f1_scores[:-1])
            thresh_pr = thresholds[best_idx]
            f1_pr = f1_scores[best_idx]
        except:
            thresh_pr = 0.5
            f1_pr = 0.0
        
        # Strategy 2: Adaptive grid search around PR threshold
        mean_prob = probs.mean()
        pos_ratio = labels.mean()
        
        # Smart search range based on class characteristics
        if pos_ratio < 0.1:  # Rare class
            search_min = max(0.01, thresh_pr - 0.1, mean_prob * 0.3)
            search_max = min(0.3, thresh_pr + 0.1, mean_prob * 2.0)
            n_points = 25
        elif pos_ratio > 0.9:  # Common class
            search_min = max(0.7, thresh_pr - 0.1, mean_prob * 0.8)
            search_max = min(0.99, thresh_pr + 0.1, mean_prob * 1.2)
            n_points = 25
        else:
            search_min = max(0.05, thresh_pr - 0.15, mean_prob * 0.5)
            search_max = min(0.95, thresh_pr + 0.15, mean_prob * 1.5)
            n_points = 20
        
        # Fine-tune with grid search
        best_f1 = f1_pr
        best_thresh = thresh_pr
        
        for thresh in np.linspace(search_min, search_max, n_points):
            preds = (probs > thresh).astype(int)
            f1 = f1_score(labels, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh
        
        # Ensure threshold is reasonable
        best_thresh = np.clip(best_thresh, 0.05, 0.95)
        
        optimal_thresholds.append(best_thresh)
        per_class_f1.append(best_f1)
    
    return optimal_thresholds, per_class_f1

In [13]:
def _ensure_shape_compatibility(y_true, logits, num_classes, device):
    """
    Ensure y_true and logits have compatible shapes
    """
    # y_true shape: (batch_size, num_classes)
    # logits shape: (batch_size, num_classes)
    
    if y_true.shape != logits.shape:
        # Try to reshape y_true to match logits
        if y_true.numel() == logits.numel():
            y_true = y_true.reshape(logits.shape)
        else:
            raise ValueError(
                f"Shape mismatch: y_true {y_true.shape} vs logits {logits.shape}"
            )
    
    return y_true

In [14]:
def train_gat(model, classifier, train_loader, val_loader, device,
              num_classes=12, epochs=100, patience=20):
    """
    Improved training function with better threshold optimization and error handling
    """
    
    print("\n" + "="*80)
    print("ANALYZING TRAINING DATA")
    print("="*80)
    
    # Calculate class statistics
    all_labels = []
    for batch in train_loader:
        batch = batch.to(device)
        y_true = extract_graph_labels(batch, num_classes, device)
        all_labels.append(y_true.cpu().numpy())
    
    all_labels = np.vstack(all_labels)
    pos_counts = all_labels.sum(axis=0)
    total_samples = len(all_labels)
    
    print("\nClass Distribution:")
    print(f"{'Class':<8} {'Samples':<10} {'Ratio':<10} {'Imbalance'}")
    print("-" * 80)
    
    for i in range(num_classes):
        ratio = pos_counts[i] / total_samples
        imbalance = (total_samples - pos_counts[i]) / (pos_counts[i] + 1e-5)
        print(f"Class {i:<2d} {int(pos_counts[i]):<10} {ratio*100:<9.2f}% {imbalance:<.2f}:1")
    
    # Setup
    model.to(device)
    classifier.to(device)
    
    # Optimizer with different learning rates
    optimizer = torch.optim.AdamW([
        {'params': model.parameters(), 'lr': 0.001, 'weight_decay': 0.01},
        {'params': classifier.parameters(), 'lr': 0.002, 'weight_decay': 0.005}
    ])
    
    # Loss function
    criterion = AsymmetricLossOptimized(
        gamma_neg=4,
        gamma_pos=1,
        clip=0.05
    )
    
    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=8, min_lr=1e-6
    )
    
    # Warmup scheduler
    warmup_scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer, start_factor=0.1, total_iters=5
    )
    
    best_f1 = 0.0
    best_epoch = 0
    patience_counter = 0
    
    print("\n" + "="*80)
    print("STARTING TRAINING")
    print("="*80)
    
    for epoch in range(epochs):
        # ===== TRAINING =====
        model.train()
        classifier.train()
        
        total_loss = 0.0
        train_probs_list = []
        
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            
            # Forward
            graph_embedding = model(batch.x, batch.edge_index, batch.batch)
            logits = classifier(graph_embedding)
            
            # Get labels
            y_true = extract_graph_labels(batch, num_classes, device)
            
            # Ensure shape compatibility
            y_true = _ensure_shape_compatibility(y_true, logits, num_classes, device)
            
            # Loss & backward
            loss = criterion(logits, y_true.float())
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(
                list(model.parameters()) + list(classifier.parameters()),
                max_norm=1.0
            )
            
            optimizer.step()
            total_loss += loss.item()
            
            # Monitor probabilities
            with torch.no_grad():
                train_probs_list.append(torch.sigmoid(logits).mean().item())
        
        # Warmup for first 5 epochs
        if epoch < 5:
            warmup_scheduler.step()
        
        avg_train_loss = total_loss / len(train_loader)
        avg_train_prob = np.mean(train_probs_list)
        
        # ===== VALIDATION =====
        model.eval()
        classifier.eval()
        
        val_loss = 0.0
        all_probs = []
        all_labels = []
        
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                
                graph_embedding = model(batch.x, batch.edge_index, batch.batch)
                logits = classifier(graph_embedding)
                probs = torch.sigmoid(logits)
                
                y_true = extract_graph_labels(batch, num_classes, device)
                y_true = _ensure_shape_compatibility(y_true, logits, num_classes, device)
                
                loss = criterion(logits, y_true.float())
                val_loss += loss.item()
                
                all_probs.append(probs.detach().cpu().numpy())
                all_labels.append(y_true.detach().cpu().numpy())
        
        all_probs = np.vstack(all_probs)
        all_labels = np.vstack(all_labels)
        
        # Find optimal thresholds
        optimal_thresholds, per_class_f1 = _find_optimal_thresholds(
            all_probs, all_labels, num_classes
        )
        
        # Apply thresholds
        all_preds = (all_probs > optimal_thresholds).astype(int)
        
        # Calculate metrics
        from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss
        
        micro_f1 = f1_score(all_labels, all_preds, average='micro', zero_division=0)
        macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
        micro_precision = precision_score(all_labels, all_preds, average='micro', zero_division=0)
        micro_recall = recall_score(all_labels, all_preds, average='micro', zero_division=0)
        hamming = hamming_loss(all_labels, all_preds)
        
        avg_val_loss = val_loss / len(val_loader)
        mean_val_prob = all_probs.mean()
        
        # Print progress
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"\n{'='*80}")
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"{'='*80}")
            print(f"Loss:        Train={avg_train_loss:.4f} | Val={avg_val_loss:.4f}")
            print(f" Mean Prob:   Train={avg_train_prob:.4f} | Val={mean_val_prob:.4f}")
            print(f" Micro F1:    {micro_f1:.4f} (P={micro_precision:.4f}, R={micro_recall:.4f})")
            print(f" Macro F1:    {macro_f1:.4f}")
            print(f" Hamming:     {hamming:.4f}")
            
            # Show class statistics
            low_f1_classes = np.argsort(per_class_f1)[:3]
            print(f"\n Lowest F1 Classes:")
            for i in low_f1_classes:
                pos_ratio = all_labels[:, i].mean()
                print(f"   Class {i}: F1={per_class_f1[i]:.3f}, "
                      f"Thresh={optimal_thresholds[i]:.3f}, "
                      f"PosRatio={pos_ratio:.3f}")
        
        # LR scheduling
        if epoch >= 5:
            scheduler.step(micro_f1)
            
            if (epoch + 1) % 10 == 0:
                current_lr = optimizer.param_groups[0]['lr']
                print(f"   Current LR: {current_lr:.6f}")
        
        # Save best model
        if micro_f1 > best_f1:
            best_f1 = micro_f1
            best_epoch = epoch
            patience_counter = 0
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'classifier_state_dict': classifier.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_f1': best_f1,
                'optimal_thresholds': optimal_thresholds,
                'mean_probability': mean_val_prob,
                'per_class_f1': per_class_f1,
                'all_probs_val': all_probs,
                'all_labels_val': all_labels
            }, 'best_model_improved.pt')
            
            print(f"\n Saved best model!")
            print(f"   Micro F1: {best_f1:.4f}")
            print(f"   Mean Prob: {mean_val_prob:.4f}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\n  Early stopping triggered at epoch {epoch+1}")
                print(f"   Best F1 was {best_f1:.4f} at epoch {best_epoch+1}")
                break
    
    print("\n" + "="*80)
    print(" TRAINING COMPLETED")
    print("="*80)
    print(f" Best Micro F1:     {best_f1:.4f}")
    print(f" Best Epoch:        {best_epoch+1}")
    print(f" Final Mean Prob:   {mean_val_prob:.4f}")
    print("="*80 + "\n")
    
    # Load best model
    checkpoint = torch.load('best_model_improved.pt', weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    classifier.load_state_dict(checkpoint['classifier_state_dict'])
    
    return model, classifier, checkpoint['optimal_thresholds']

In [15]:
from sklearn.metrics import (
    accuracy_score, f1_score, hamming_loss, jaccard_score,
    precision_score, recall_score, classification_report
)
def evaluate_multilabel(model, classifier, data_loader, device, 
                         num_classes=12, optimal_thresholds=None):
    """
    Fixed evaluation function - no more RuntimeError!
    """
    
    model.eval()
    classifier.eval()
    
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for batch in data_loader:
            batch = batch.to(device)
            
            graph_embedding = model(batch.x, batch.edge_index, batch.batch)
            logits = classifier(graph_embedding)
            probs = torch.sigmoid(logits)
            
            y_true = extract_graph_labels(batch, num_classes, device)
            
            if y_true.shape != logits.shape:
                y_true = y_true.view(logits.shape[0], -1)
                if y_true.shape[1] < num_classes:
                    pad = num_classes - y_true.shape[1]
                    y_true = torch.cat([
                        y_true,
                        torch.zeros(y_true.shape[0], pad, device=device)
                    ], dim=1)
                elif y_true.shape[1] > num_classes:
                    y_true = y_true[:, :num_classes]
            
            # FIX: Add .detach()
            all_probs.append(probs.detach().cpu().numpy())
            all_labels.append(y_true.detach().cpu().numpy())
    
    all_probs = np.vstack(all_probs)
    all_labels = np.vstack(all_labels)
    
    # Apply thresholds
    if optimal_thresholds is None:
        optimal_thresholds = [0.05] * num_classes
    
    all_preds = np.zeros_like(all_probs)
    for i in range(num_classes):
        all_preds[:, i] = (all_probs[:, i] > optimal_thresholds[i]).astype(int)
    
    # Metrics
    from sklearn.metrics import accuracy_score
    micro_f1 = f1_score(all_labels, all_preds, average='micro', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    micro_p = precision_score(all_labels, all_preds, average='micro', zero_division=0)
    micro_r = recall_score(all_labels, all_preds, average='micro', zero_division=0)
    hamming = hamming_loss(all_labels, all_preds)
    subset_acc = accuracy_score(all_labels, all_preds)
    
    per_class_f1 = f1_score(all_labels, all_preds, average=None, zero_division=0)
    per_class_p = precision_score(all_labels, all_preds, average=None, zero_division=0)
    per_class_r = recall_score(all_labels, all_preds, average=None, zero_division=0)
    
    print("\n" + "="*80)
    print("FINAL EVALUATION RESULTS")
    print("="*80)
    print(f"\n Overall Metrics:")
    print(f"   Micro F1:        {micro_f1:.4f}")
    print(f"   Macro F1:        {macro_f1:.4f}")
    print(f"   Micro Precision: {micro_p:.4f}")
    print(f"   Micro Recall:    {micro_r:.4f}")
    print(f"   Hamming Loss:    {hamming:.4f}")
    print(f"   Subset Accuracy: {subset_acc:.4f}")
    
    print(f"\n Per-Class Results:")
    print(f"{'Class':<8} {'Threshold':<12} {'F1':<10} {'Precision':<12} {'Recall':<10} {'MeanProb':<10}")
    print("-" * 80)
    
    class_names = [
        "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase",
        "NR-ER", "NR-ER-LBD", "NR-PPAR-γ",
        "SR-ARE", "SR-ATAD5", "SR-HSE", "SR-MMP", "SR-p53"
    ]
    
    for i in range(num_classes):
        name = class_names[i] if i < len(class_names) else f"Class{i}"
        print(f"{name:<8} {optimal_thresholds[i]:<12.3f} "
              f"{per_class_f1[i]:<10.3f} {per_class_p[i]:<12.3f} "
              f"{per_class_r[i]:<10.3f} {all_probs[:, i].mean():<10.3f}")
    
    print("="*80 + "\n")
    
    return {
        'micro_f1': micro_f1,
        'macro_f1': macro_f1,
        'predictions': all_preds,
        'probabilities': all_probs,
        'labels': all_labels
    }



# Embedding

In [16]:
from tqdm import tqdm
import torch
import os

def extract_embeddings(model, classifier, data_loader, device, 
                       num_classes=12, save_path='embeddings.pt'):

    print("\n" + "="*80)
    print("EXTRACTING GRAPH EMBEDDINGS (with mol_id)")
    print("="*80)

    model.to(device).eval()
    classifier.to(device).eval()

    all_embeddings = []
    all_logits = []
    all_probs = []
    all_labels = []
    all_mol_ids = []   # <-- lưu mol_id thật

    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(data_loader)):

            batch = batch.to(device)

            # ==== (1) Kiểm tra mol_id có tồn tại không ====
            if not hasattr(batch, "mol_id"):
                raise ValueError(
                    "batch không có mol_id! "
                    "Hãy chắc chắn bạn có Data(mol_id=...) trong dataset."
                )

            # ==== (2) Xử lý mol_id sau batching ====
            # PyG chuyển thành list khi batch
            if isinstance(batch.mol_id, list):
                batch_mol_ids = batch.mol_id
            else:
                # nếu tensor → convert sang list
                if torch.is_tensor(batch.mol_id):
                    batch_mol_ids = batch.mol_id.cpu().tolist()
                else:
                    # string/dơn item
                    batch_mol_ids = [batch.mol_id]

            all_mol_ids.extend(batch_mol_ids)

            # ==== (3) Lấy graph embedding ====
            graph_embedding = model(batch.x, batch.edge_index, batch.batch)

            # ==== (4) Classifier ====
            logits = classifier(graph_embedding)
            probs = torch.sigmoid(logits)

            # ==== (5) Labels ====
            y_true = extract_graph_labels(batch, num_classes, device)

            # reshape nếu mismatch
            if y_true.shape != logits.shape:
                y_true = y_true.view(logits.shape[0], -1)

            # ==== (6) Lưu vào list ====
            all_embeddings.append(graph_embedding.cpu())
            all_logits.append(logits.cpu())
            all_probs.append(probs.cpu())
            all_labels.append(y_true.cpu())

    # ==== CONCAT ====
    embeddings = torch.cat(all_embeddings, dim=0)
    logits      = torch.cat(all_logits, dim=0)
    probs       = torch.cat(all_probs, dim=0)
    labels      = torch.cat(all_labels, dim=0)

    print(f"\n Extracted {embeddings.shape[0]} graph embeddings")
    print(f" Embedding dim: {embeddings.shape[1]}")

    # ==== SAVE ====
    output = {
        "embeddings": embeddings,
        "logits": logits,
        "probabilities": probs,
        "labels": labels,
        "mol_ids": all_mol_ids,     # <-- lưu mol_id thật
        "num_graphs": embeddings.shape[0],
        "embedding_dim": embeddings.shape[1],
        "num_classes": num_classes,
    }

    torch.save(output, save_path)

    print(f"\n Saved to {save_path}")
    print(f" File size: {os.path.getsize(save_path)/(1024*1024):.2f} MB")
    print("="*80)

    return output


In [17]:
def extract_embeddings_by_split(model, classifier, train_loader, val_loader, 
                                test_loader, device, num_classes=12, 
                                output_dir='embeddings'):
    """
    Extract embeddings cho tất cả các splits (train/val/test) và lưu riêng
    
    Args:
        model: Trained GAT model
        classifier: Trained classifier
        train_loader, val_loader, test_loader: DataLoaders
        device: torch device
        num_classes: Số lượng classes
        output_dir: Thư mục output
    
    Returns:
        Dictionary chứa paths đến các embedding files
    """
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    paths = {}
    
    # Extract train embeddings
    if train_loader is not None:
        print("\n🔹 Extracting TRAIN embeddings...")
        train_path = os.path.join(output_dir, 'train_embeddings.pt')
        extract_embeddings(model, classifier, train_loader, device, 
                          num_classes, train_path)
        paths['train'] = train_path
    
    # Extract validation embeddings
    if val_loader is not None:
        print("\n🔹 Extracting VALIDATION embeddings...")
        val_path = os.path.join(output_dir, 'val_embeddings.pt')
        extract_embeddings(model, classifier, val_loader, device, 
                          num_classes, val_path)
        paths['val'] = val_path
    
    # Extract test embeddings
    if test_loader is not None:
        print("\n🔹 Extracting TEST embeddings...")
        test_path = os.path.join(output_dir, 'test_embeddings.pt')
        extract_embeddings(model, classifier, test_loader, device, 
                          num_classes, test_path)
        paths['test'] = test_path
    
    print("\n" + "="*80)
    print(" ALL EMBEDDINGS EXTRACTED SUCCESSFULLY")
    print("="*80)
    for split, path in paths.items():
        print(f"   {split.upper()}: {path}")
    print("="*80 + "\n")
    
    return paths


In [20]:
# ============================================================================
# INSPECT ALL ATTRIBUTES IN DATASET
# ============================================================================

print("="*80)
print("INSPECTING DATA ATTRIBUTES")
print("="*80)

sample = train_dataset[0]
print(f"\nFirst sample attributes and types:")

for key in sample.keys():
    value = sample[key]
    if isinstance(value, torch.Tensor):
        print(f"  {key}: Tensor {value.shape} dtype={value.dtype}")
    elif isinstance(value, (int, float, str)):
        print(f"  {key}: {type(value).__name__} = {value}")
    else:
        print(f"  {key}: {type(value).__name__}")

# Check a few more samples to see if attributes vary
print(f"\nChecking attribute consistency across 5 samples:")
for i in range(min(5, len(train_dataset))):
    data = train_dataset[i]
    print(f"\nSample {i}:")
    for key in data.keys():
        value = data[key]
        if isinstance(value, torch.Tensor):
            print(f"  {key}: {value.shape}")
        else:
            print(f"  {key}: {value}")

# ============================================================================
# CLEAN DATASET - REMOVE/STANDARDIZE PROBLEMATIC ATTRIBUTES
# ============================================================================

print("\n" + "="*80)
print("CLEANING DATASET")
print("="*80)

def clean_dataset(dataset, name):
    """Remove problematic attributes from dataset"""
    print(f"\nCleaning {name}...")
    
    for i, data in enumerate(dataset):
        # Keep only essential attributes
        keys_to_keep = ['x', 'edge_index', 'y', 'mask', 'mol_id']
        keys_to_remove = [k for k in data.keys() if k not in keys_to_keep]
        
        for key in keys_to_remove:
            delattr(data, key)
    
    print(f"   Kept only: {keys_to_keep}")
    print(f"   Removed: {keys_to_remove}")

clean_dataset(train_dataset, "train_dataset")
clean_dataset(val_dataset, "val_dataset")
clean_dataset(test_dataset, "test_dataset")

print("\n Dataset cleaned!")

# Verify
sample = train_dataset[0]
print(f"\nAfter cleaning, attributes: {list(sample.keys())}")

# ============================================================================
# SIMPLE COLLATE FUNCTION
# ============================================================================

print("\n" + "="*80)
print("CREATING SIMPLE COLLATE FUNCTION")
print("="*80)

from torch.utils.data import DataLoader
from torch_geometric.data import Batch
import torch

def simple_collate_fn(data_list):
    """
    Simple collate - only concatenate x, edge_index
    Reshape y, mask manually
    """
    batch = Batch.from_data_list(data_list)
    
    batch_size = len(data_list)
    num_classes = 12
    
    # Reshape y: (batch_size * 12,) → (batch_size, 12)
    if batch.y is not None:
        batch.y = batch.y.reshape(batch_size, num_classes)
    
    # Reshape mask: (batch_size * 12,) → (batch_size, 12)
    if batch.mask is not None:
        batch.mask = batch.mask.reshape(batch_size, num_classes)
    
    return batch


# ============================================================================
# CREATE DATALOADERS
# ============================================================================

print("\nCreating dataloaders...\n")

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=simple_collate_fn
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=simple_collate_fn
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=simple_collate_fn
)

print(f" train_loader: {len(train_loader)} batches")
print(f" val_loader: {len(val_loader)} batches")
print(f" test_loader: {len(test_loader)} batches")

# ============================================================================
# VERIFY DATALOADERS
# ============================================================================

print("\n" + "="*80)
print("VERIFYING DATALOADERS")
print("="*80)

success = True
try:
    for batch_idx, batch in enumerate(train_loader):
        print(f"\nBatch {batch_idx}:")
        print(f"  x shape:         {batch.x.shape}")
        print(f"  edge_index shape: {batch.edge_index.shape}")
        print(f"  y shape:         {batch.y.shape}")
        print(f"  mask shape:      {batch.mask.shape}")
        
        # Verify shapes
        assert batch.y.shape == (32, 12), f"y shape wrong: {batch.y.shape}"
        assert batch.mask.shape == (32, 12), f"mask shape wrong: {batch.mask.shape}"
        
        if batch_idx >= 2:
            break
    
    print(f"\n" + "="*80)
    print(f" SUCCESS! DATALOADERS READY FOR TRAINING!")
    print(f"="*80 + "\n")
    
except Exception as e:
    print(f"\n ERROR: {e}")
    import traceback
    traceback.print_exc()
    success = False



INSPECTING DATA ATTRIBUTES

First sample attributes and types:
  mol_id: str = TOX9302
  mask: Tensor torch.Size([12]) dtype=torch.float32
  x: Tensor torch.Size([11, 41]) dtype=torch.float32
  edge_index: Tensor torch.Size([2, 20]) dtype=torch.int64
  y: Tensor torch.Size([12]) dtype=torch.float32

Checking attribute consistency across 5 samples:

Sample 0:
  mol_id: TOX9302
  mask: torch.Size([12])
  x: torch.Size([11, 41])
  edge_index: torch.Size([2, 20])
  y: torch.Size([12])

Sample 1:
  mol_id: TOX25748
  mask: torch.Size([12])
  x: torch.Size([20, 41])
  edge_index: torch.Size([2, 40])
  y: torch.Size([12])

Sample 2:
  mol_id: TOX4625
  mask: torch.Size([12])
  x: torch.Size([16, 41])
  edge_index: torch.Size([2, 34])
  y: torch.Size([12])

Sample 3:
  mol_id: TOX22180
  mask: torch.Size([12])
  x: torch.Size([13, 41])
  edge_index: torch.Size([2, 26])
  y: torch.Size([12])

Sample 4:
  mol_id: TOX4559
  mask: torch.Size([12])
  x: torch.Size([16, 41])
  edge_index: torch.Size

In [21]:

print(f"   Train: {len(train_dataset)} samples")
print(f"   Val:   {len(val_dataset)} samples")
print(f"   Test:  {len(test_dataset)} samples\n")

# ============================================================================
# SETUP DEVICE
# ============================================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f" Device: {device}\n")


model = GAT(
    input_dim=41,          # số đặc trưng input của node
    hidden_dim=128,       # hidden dimension
    num_heads=4,          # số attention heads
    num_layers=3,         # số layer GAT
    embedding_dim=512,    # kích thước embedding cuối
    dropout=0.2
)

# 2. Classifier (FIX: Chỉ khởi tạo 1 lần)
classifier = MultiLabelClassifier(
    embedding_dim=512,
    num_classes=12,
    hidden_dim=256,
    dropout=0.3,
    temperature=1.0
)

print(f" GAT parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f" Classifier parameters: {sum(p.numel() for p in classifier.parameters()):,}\n")

# ============================================================================
# TRAINING
# ============================================================================
print(" Starting training...\n")

model, classifier, optimal_thresholds = train_gat(
    model=model,
    classifier=classifier,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    num_classes=12,
    epochs=200,
    patience=20
)

# ============================================================================
# EVALUATION ON TEST SET
# ============================================================================
print("\n Evaluating on test set...")

test_results = evaluate_multilabel(
    model=model,
    classifier=classifier,
    data_loader=test_loader,
    device=device,
    num_classes=12,
    optimal_thresholds=optimal_thresholds
)

# Print final results
print("\n" + "="*80)
print(" FINAL TEST RESULTS")
print("="*80)
print(f"Micro F1:       {test_results['micro_f1']:.4f}")
print(f"Macro F1:       {test_results['macro_f1']:.4f}")




print("="*80 + "\n")

# ============================================================================
# EXTRACT EMBEDDINGS (OPTIONAL)
# ============================================================================
print(" Extracting embeddings from trained model...")

embeddings_paths = extract_embeddings_by_split(
    model=model,
    classifier=classifier,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    device=device,
    num_classes=12,
    output_dir='embeddings'
)

   Train: 6404 samples
   Val:   801 samples
   Test:  801 samples

 Device: cuda

 GAT parameters: 1,560,448
 Classifier parameters: 166,541

 Starting training...


ANALYZING TRAINING DATA

Class Distribution:
Class    Samples    Ratio      Imbalance
--------------------------------------------------------------------------------
Class 0  254        3.97     % 24.21:1
Class 1  202        3.15     % 30.70:1
Class 2  626        9.78     % 9.23:1
Class 3  237        3.70     % 26.02:1
Class 4  633        9.88     % 9.12:1
Class 5  295        4.61     % 20.71:1
Class 6  154        2.40     % 40.58:1
Class 7  763        11.91    % 7.39:1
Class 8  215        3.36     % 28.79:1
Class 9  301        4.70     % 20.28:1
Class 10 754        11.77    % 7.49:1
Class 11 356        5.56     % 16.99:1

STARTING TRAINING


/usr/local/lib/python3.11/dist-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(



Epoch 1/200
Loss:        Train=0.0999 | Val=0.0508
 Mean Prob:   Train=0.4419 | Val=0.4186
 Micro F1:    0.2241 (P=0.1417, R=0.5356)
 Macro F1:    0.2264
 Hamming:     0.2385

 Lowest F1 Classes:
   Class 6: F1=0.111, Thresh=0.507, PosRatio=0.021
   Class 9: F1=0.117, Thresh=0.455, PosRatio=0.056
   Class 3: F1=0.121, Thresh=0.518, PosRatio=0.037

 Saved best model!
   Micro F1: 0.2241
   Mean Prob: 0.4186


/usr/local/lib/python3.11/dist-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(



 Saved best model!
   Micro F1: 0.3072
   Mean Prob: 0.4055


...


 Extracted 6404 graph embeddings
 Embedding dim: 512

 Saved to embeddings/train_embeddings.pt
 File size: 13.50 MB

🔹 Extracting VALIDATION embeddings...

EXTRACTING GRAPH EMBEDDINGS (with mol_id)


100%|██████████| 26/26 [00:00<00:00, 158.10it/s]



 Extracted 801 graph embeddings
 Embedding dim: 512

 Saved to embeddings/val_embeddings.pt
 File size: 1.69 MB

🔹 Extracting TEST embeddings...

EXTRACTING GRAPH EMBEDDINGS (with mol_id)


100%|██████████| 26/26 [00:00<00:00, 158.59it/s]


 Extracted 801 graph embeddings
 Embedding dim: 512

 Saved to embeddings/test_embeddings.pt
 File size: 1.69 MB

 ALL EMBEDDINGS EXTRACTED SUCCESSFULLY
   TRAIN: embeddings/train_embeddings.pt
   VAL: embeddings/val_embeddings.pt
   TEST: embeddings/test_embeddings.pt



In [22]:
path = "/kaggle/working/embeddings/train_embeddings.pt"

data = torch.load(path)
print(data.keys())
print(data['mol_ids'][:10])

dict_keys(['embeddings', 'logits', 'probabilities', 'labels', 'mol_ids', 'num_graphs', 'embedding_dim', 'num_classes'])
['TOX27328', 'TOX29127', 'TOX31604', 'TOX4368', 'TOX5599', 'TOX25380', 'TOX27453', 'TOX23895', 'TOX2659', 'TOX1892']
